In [5]:
pip install requests

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import requests
import pandas as pd

def get_nutrition(food_name):
    nutrition_data = pd.DataFrame(columns=['name', 'protein', 'calcium', 'fat', 'carbohydrates', 'vitamins'])
    for name in food_name:
        url = "https://api.nal.usda.gov/fdc/v1/foods/search?api_key=SJdmyebqjsKZvKvtLr79kLjAvrIeGusyiibksZaz&query=" + name
        response = requests.get(url)
        data = response.json()
        flatten_json = pd.json_normalize(data["foods"])
        if not flatten_json.empty:  # Check if flatten_json is not empty
            first_food = flatten_json.iloc[0]
            first_food_nutrition_list = first_food.foodNutrients
            protein = next((item['value'] for item in first_food_nutrition_list if item['nutrientNumber'] == "203"), 0)
            calcium = next((item['value'] for item in first_food_nutrition_list if item['nutrientNumber'] == "301"), 0)
            fat = next((item['value'] for item in first_food_nutrition_list if item['nutrientNumber'] == "204"), 0)
            carbs = next((item['value'] for item in first_food_nutrition_list if item['nutrientNumber'] == "205"), 0)
            vitamin_a = next((item['value'] for item in first_food_nutrition_list if item['nutrientNumber'] == "318"), 0)
            vitamin_c = next((item['value'] for item in first_food_nutrition_list if item['nutrientNumber'] == "401"), 0)
            vitamins = float(vitamin_a) + float(vitamin_c)
            nutrition_data = pd.concat([nutrition_data, pd.DataFrame({
                'name': [name],
                'protein': [protein],
                'calcium': [calcium / 1000],
                'fat': [fat],
                'carbohydrates': [carbs],
                'vitamins': [vitamins / 1000]
            })], ignore_index=True)
        else:
            print(f"No data found for {name}")

    return nutrition_data

food_items = ['apple_pie', 'bibimbap', 'cannoli', 'edamame', 'falafel', 'french_toast', 'ice_cream', 'ramen', 'sushi', 'tiramisu']

nutrition_data = get_nutrition(food_items)
nutrition_data.to_csv("nutrition_infor.csv", index=False)


No data found for apple_pie


C:\Users\aditi\AppData\Local\Temp\ipykernel_284\4045481570.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  nutrition_data = pd.concat([nutrition_data, pd.DataFrame({


No data found for french_toast
No data found for ice_cream


In [1]:
import requests
import pandas as pd

# Your API key from the USDA FoodData Central website
api_key = 'SJdmyebqjsKZvKvtLr79kLjAvrIeGusyiibksZaz'

# Base URL for the API
base_url = 'https://api.nal.usda.gov/fdc/v1/'

# Function to get nutritional information for a food item
def get_nutrition(food_name):
    url = base_url + f"foods/search?api_key={api_key}&query={food_name}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if 'foods' in data and data['foods']:
            food = data['foods'][0]
            nutrients = food.get('foodNutrients', [])
            nutrition_info = {
                'name': food_name,
                'protein': next((nutrient['value'] for nutrient in nutrients if nutrient['nutrientName'] == 'Protein'), 0),
                'calcium': next((nutrient['value'] for nutrient in nutrients if nutrient['nutrientName'] == 'Calcium, Ca'), 0),
                'fat': next((nutrient['value'] for nutrient in nutrients if nutrient['nutrientName'] == 'Total lipid (fat)'), 0),
                'carbohydrates': next((nutrient['value'] for nutrient in nutrients if nutrient['nutrientName'] == 'Carbohydrate, by difference'), 0),
                'vitamin_a': next((nutrient['value'] for nutrient in nutrients if nutrient['nutrientName'] == 'Vitamin A, IU'), 0),
                'vitamin_c': next((nutrient['value'] for nutrient in nutrients if nutrient['nutrientName'] == 'Vitamin C, total ascorbic acid'), 0)
            }
            return nutrition_info
    print(f"No data found for {food_name}")
    return None

# List of food items
food_items = ['apple_pie', 'bibimbap', 'cannoli', 'edamame', 'falafel', 'french_toast', 'ice_cream', 'ramen', 'sushi', 'tiramisu']

# List to store the nutritional information for each food item
nutrition_data = []

# Retrieve nutritional information for each food item
for food_item in food_items:
    nutrition_info = get_nutrition(food_item)
    if nutrition_info:
        nutrition_data.append(nutrition_info)

# Create a DataFrame from the nutrition data
nutrition_df = pd.DataFrame(nutrition_data)

# Save the DataFrame to a CSV file
nutrition_df.to_csv('nutrition_infor.csv', index=False)


No data found for apple_pie
No data found for french_toast
No data found for ice_cream
